## this notebook is to compare repsonses from base and instruct models

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
def load_model():
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
    return model, tokenizer

model, tokenizer = load_model()

In [3]:
def load_model_base():
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B")
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
    return model, tokenizer

model_base, tokenizer_base = load_model_base()


In [4]:
def stream_instruct_response(instruction="What is the capital city of U.S. "):
    # Apply chat template to format the prompt correctly
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction}
    ]

    # Use apply_chat_template to format the messages properly
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        text,
        add_special_tokens=False,
        return_tensors="pt",
    )

    from transformers import TextStreamer

    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=32,
        use_cache=True,
        temperature=0.7,
        min_p=0.1,
    )


In [ ]:
def stream_base_response(instruction):
    # For base model, use the instruction directly without chat template
    text = instruction

    inputs = tokenizer_base(
        text,
        add_special_tokens=True,
        return_tensors="pt",
    )

    from transformers import TextStreamer

    text_streamer = TextStreamer(tokenizer_base, skip_prompt=True)
    _ = model_base.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=32,
        use_cache=True,
        temperature=0.7,
        pad_token_id=tokenizer_base.eos_token_id,
    )

---

In [6]:
prompt = "What is the capital city of U.S. "


In [7]:
stream_instruct_response(prompt)

The capital city of the United States is Washington, D.C.<|im_end|>


In [12]:
stream_base_response(prompt)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


101?
The capital city of U.S. 101 is San Francisco, California.<|endoftext|>
